In [85]:
#import the pandas library
try:
    import pandas as pd
except: 
    !pip install pandas --quiet
    import pandas as pd
# #import the sklearn-preprocessing library
try:
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
except:
    !pip install -U scikit-learn --quiet
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import MinMaxScaler
    from sklearn.model_selection import train_test_split
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report
#import the pickle library
try:
    import pickle as pkl
except:
    !pip install pickle --quiet
    import pickle as pkl

In [34]:
#load in training data
train_df = pd.read_csv('train.csv')

In [1]:

try:
    from dataprep.eda import create_report
except:
    !pip install -U dataprep.eda
    # !pip install dataprep.eda --quiet
    from dataprep.eda import create_report

# eda_report = create_report(train_df, title = "Kaggle: Spaceship Titanic EDA report")
# pkl.dump(eda_report, 'SpaceshipEDA.pkl')

ERROR: Could not find a version that satisfies the requirement dataprep.eda (from versions: none)
ERROR: No matching distribution found for dataprep.eda


ModuleNotFoundError: No module named 'dataprep'

In [4]:
pip list

Package                   Version
------------------------- -----------
anyio                     4.2.0
argon2-cffi               21.3.0
argon2-cffi-bindings      21.2.0
asttokens                 2.0.5
async-lru                 2.0.4
attrs                     23.1.0
Babel                     2.11.0
beautifulsoup4            4.12.3
bleach                    4.1.0
Brotli                    1.0.9
certifi                   2024.6.2
cffi                      1.16.0
charset-normalizer        2.0.4
colorama                  0.4.6
comm                      0.2.1
debugpy                   1.6.7
decorator                 5.1.1
defusedxml                0.7.1
executing                 0.8.3
fastjsonschema            2.16.2
idna                      3.7
ipykernel                 6.28.0
ipython                   8.25.0
jedi                      0.18.1
Jinja2                    3.1.4
joblib                    1.4.2
json5                     0.9.6
jsonschema                4.19.2
jsonschema-specifica

In [35]:
#intial length of dataset
print(f"intial length of dataset {len(train_df)}",'\n')

#remove columns that are highly unqiue (name and ID)
train_df=train_df.drop(columns=["Name","PassengerId"])

#handle duplicate rows
#check intial number of duplicates
print(f"intial num dups: {train_df.duplicated().sum()}")
#remove duplicates
train_df = train_df.drop_duplicates()
#show the new number of duplicates
print(f"new num dups: {train_df.duplicated().sum()}",'\n')

#handle NaN values
#check intial number of NaN
print(f"intial num NaN:{train_df.isna().sum()}")
#remove NaN values
train_df.dropna(inplace=True)
#show new number of NaN values
print(f"new num NaN:{train_df.isna().sum()}",'\n')

#fix index on dataframe
train_df.reset_index(drop=True)

#new length of dataset
print(f"new length of dataset {len(train_df)}",'\n')

intial length of dataset 8693 

intial num dups: 15
new num dups: 0 

intial num NaN:HomePlanet      201
CryoSleep       217
Cabin           193
Destination     182
Age             179
VIP             203
RoomService     181
FoodCourt       183
ShoppingMall    208
Spa             183
VRDeck          188
Transported       0
dtype: int64
new num NaN:HomePlanet      0
CryoSleep       0
Cabin           0
Destination     0
Age             0
VIP             0
RoomService     0
FoodCourt       0
ShoppingMall    0
Spa             0
VRDeck          0
Transported     0
dtype: int64 

new length of dataset 6755 



In [5]:
#current dataset (first 5 rows)
train_df.head()

,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
0,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,False
1,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,True
2,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,False
3,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,False
4,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,True


Transform data

In [36]:
#turns a binary value into a 1 or 0
def to_binary(df,column):
    #set false to be 0 and true to be 1
    mapping = {False: 0, True: 1}
    #change dataframe to reflect the numerical (binary) representation
    df[column] = df[column].map(mapping)
    return df
#binary columns in dataset
binary_columns = [
    "CryoSleep", #turn cryp sleep 1 and 0
    "VIP", #turn vip to 1 and 0
    "Transported" #turn transported into 1 and 0
]
#goes through all binary columns in dataset and changes them to 1 and 0 in are data
for column in binary_columns:
    train_df = to_binary(train_df,column)


In [37]:
#combine money spent across all services into 1 column
#all the luxury services that cost money in the data
luxury_services = [
    "RoomService",
    "FoodCourt",
    "ShoppingMall",
    "Spa",
    "VRDeck"
]
#added all money spent and create new column "money_spent" 
train_df["money_spent"] = train_df[luxury_services].sum(axis=1)
#remove all of the luxury services columns
train_df = train_df.drop(columns = luxury_services)

In [39]:
#split cabin into it three respective parts of deck, number and side
train_df[["deck","cabin_number","side"]] = train_df["Cabin"].str.split('/',expand=True)
train_df["side"]=train_df["side"].map({"P":0,"S":1})
train_df = train_df.drop(columns = "Cabin")

In [55]:
#show data after transformation
train_df.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,Transported,money_spent,deck,cabin_number,side
0,Europa,0,TRAPPIST-1e,39.0,0,0,0.0,B,0,0
1,Earth,0,TRAPPIST-1e,24.0,0,1,736.0,F,0,1
2,Europa,0,TRAPPIST-1e,58.0,1,0,10383.0,A,0,1
3,Europa,0,TRAPPIST-1e,33.0,0,0,5176.0,A,0,1
4,Earth,0,TRAPPIST-1e,16.0,0,1,1091.0,F,1,1


In [76]:
#seperate features and label
feature_names = [
    "HomePlanet",
    "CryoSleep",
    "Destination",
    "Age",
    "VIP",
    "money_spent",
    "deck",
    "cabin_number",
    "side"
]
label_name = "Transported"
#seperate training and validation data
train_features, val_features, train_label, val_label=train_test_split(train_df[features],train_df[label], test_size=0.2, random_state=42)
#save all datasets
pkl.dump(train_features,open("train_features.pkl","wb"))
pkl.dump(train_label,open("train_label.pkl","wb"))
pkl.dump(val_features,open("val_features.pkl","wb"))
pkl.dump(val_label,open("val_label.pkl","wb"))

In [77]:
#load saved datasets
train_features  =pkl.load(open("train_features.pkl","rb"))
train_label = pkl.load(open("train_label.pkl","rb"))
val_features = pkl.load(open("val_features.pkl","rb"))
val_label = pkl.load(open("val_label.pkl","rb"))

In [78]:
train_features.head()

,HomePlanet,CryoSleep,Destination,Age,VIP,money_spent,deck,cabin_number,side
3099,Earth,1,PSO J318.5-22,28.0,0,0.0,G,546,0
5868,Europa,0,55 Cancri e,37.0,0,3383.0,B,239,1
7888,Earth,1,TRAPPIST-1e,20.0,0,0.0,G,1349,1
3929,Earth,1,TRAPPIST-1e,13.0,0,0.0,G,688,1
5239,Mars,1,55 Cancri e,53.0,0,0.0,F,1161,0


Normalization

In [79]:
#onehot encode the nominal features in the dataset

#nominal features in dataset to be one hot encoded
nominal_columns = [
    "HomePlanet", #onehot encode homeplanet
    "Destination", #onehot encode destination
    "deck"
]

#fit onehot encoder to nominal data
def fit_one_hot_encoding(df,nominal_features):
    encoder = OneHotEncoder(handle_unknown='ignore')
    #fit encoder on training data
    encoder.fit(df[nominal_features])
    #save fitted onehot encoder
    pkl.dump(encoder,open("OneHotEncoder.pkl","wb"))

#transform nominal data to one hot encoding with encoder
def to_onehot(df,nominal_features,encoder_file):
    #load existing encoder 
    encoder = pkl.load(open(encoder_file,"rb"))
    #transform nominal features to onehot
    onehot = encoder.transform(df[nominal_features])
    #create matrix of nominal feature
    onehot_df = pd.DataFrame(onehot.todense(), columns = encoder.get_feature_names_out())
    #update dataframe
    df = df.drop(columns=nominal_features)
    df.reset_index(drop=True, inplace=True)
    df = pd.concat([df,onehot_df],axis=1)
    #save changes and return dataset
    return df

fit_one_hot_encoding(train_features, nominal_columns)
train_features = to_onehot(train_features,nominal_columns,"OneHotEncoder.pkl")

In [80]:
#minmax scale numerical features in the datasett

#numerical features in the dataset to be encoded
numerical_columns = [
    "Age", #minmax age
    "money_spent", #minmax money_spent
    "cabin_number"
]

#fit scaler to numerical data
def fit_MinMax(df,numerical_features):
    Scaler = MinMaxScaler()
    #fit scaler on training data
    Scaler.fit(df[numerical_features])
    #save fitted scaler
    pkl.dump(Scaler,open("MinMaxScaler.pkl","wb"))

def to_MinMax(df,numerical_features,scaler_file):
    #load existing scaler 
    scaler = pkl.load(open(scaler_file,"rb"))
    #transform numerical features to MinMax
    df[numerical_features] = scaler.transform(df[numerical_features])
    #save changes and return dataset
    return df

fit_MinMax(train_features,numerical_columns)
train_features = to_MinMax(train_features,numerical_columns,"MinMaxScaler.pkl")

In [81]:
norm_train_features = train_features
norm_train_features.head()

,CryoSleep,Age,VIP,money_spent,cabin_number,side,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,deck_A,deck_B,deck_C,deck_D,deck_E,deck_F,deck_G,deck_T
0,1,0.354430,0,0.000000,0.288736,0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0,0.468354,0,0.094006,0.126388,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,0.253165,0,0.000000,0.713379,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,1,0.164557,0,0.000000,0.363829,1,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,1,0.670886,0,0.000000,0.613961,0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [82]:
#train random forest
clf = RandomForestClassifier(random_state=42)
clf.fit(norm_train_features,train_label)

RandomForestClassifier(random_state=42)

In [88]:
val_features = to_onehot(val_features,nominal_columns,"OneHotEncoder.pkl")
val_features = to_MinMax(val_features,numerical_columns,"MinMaxScaler.pkl")
norm_val_features = val_features

In [89]:
label_pred = clf.predict(norm_val_features)
print(classification_report(val_label,label_pred))

              precision    recall  f1-score   support

           0       0.72      0.83      0.77       664
           1       0.81      0.69      0.75       687

    accuracy                           0.76      1351
   macro avg       0.77      0.76      0.76      1351
weighted avg       0.77      0.76      0.76      1351

